# Cleaning Data

The data following the data fusion process still had a number of problems. Primarily there were a number of null values across the columns and rows. Additionally, data was not available for the BAH rates for 1997, which means that there was nothing to merge onto for that year.

To handle these issues we proceed though the following steps:

1. Perform some data exploration to see the extent of the missing data values.
2. Determine what data can/should be dropped.
3. Perform imputation on any missing data values that should be filled.
4. Merge the final set of data together.

*Package Imports*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

*Global Variables/Settings*

In [2]:
raw_dir = "Raw Data"
fuse_dir = "Fused Data"
clean_dir = "Clean Data"

## 1.(a) Data Exploration

In [3]:
# import the fused data set
fused_file = os.path.join(fuse_dir,"fused_df.csv")
base_df = pd.read_csv(fused_file)
base_df.head()

,Dependents,Year,MHA,E1,E2,E3,E4,E5,E6,E7,...,ZIP,Latitude,Longitude,CITY,STATE,Average CPI,Senate Majority Party,House Majority Party,President Party,Index (NSA)
0,1,1995,AK404,343.72,336.38,342.40,389.17,478.92,545.71,612.63,...,99505,61.266470,-149.641223,ANCHORAGE,AK,152.383333,0,0,0,100.0
1,1,1995,AK404,343.72,336.38,342.40,389.17,478.92,545.71,612.63,...,99505,61.266470,-149.641223,ANCHORAGE,AK,152.383333,1,1,0,100.0
2,0,1995,AK404,192.42,211.35,252.19,271.72,334.33,371.40,425.62,...,99505,61.266470,-149.641223,ANCHORAGE,AK,152.383333,0,0,0,100.0
3,0,1995,AK404,192.42,211.35,252.19,271.72,334.33,371.40,425.62,...,99505,61.266470,-149.641223,ANCHORAGE,AK,152.383333,1,1,0,100.0
4,1,1995,AK405,257.40,257.40,262.27,313.24,375.78,413.67,444.98,...,99701,64.864904,-146.775162,FAIRBANKS,AK,152.383333,0,0,0,NaN


In [5]:
# get the dimensions of the dataframe
base_df.shape

(47976, 40)

In [4]:
# use the describe function to get a clearer picture of the dataframe
base_df.describe(include="all")

,Dependents,Year,MHA,E1,E2,E3,E4,E5,E6,E7,...,ZIP,Latitude,Longitude,CITY,STATE,Average CPI,Senate Majority Party,House Majority Party,President Party,Index (NSA)
count,47976.000000,47976.000000,47976,47930.000000,47930.000000,47930.000000,47930.000000,47930.000000,47930.000000,47930.000000,...,47976.000000,47976.000000,47976.000000,47976,47976,47976.000000,47976.000000,47976.000000,47976.000000,46962.000000
unique,NaN,NaN,121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,121,42,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,ZZ510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,COUNTY COST GROUP 510,CCG,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,9468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9468,26232,NaN,NaN,NaN,NaN,NaN
mean,0.500000,2009.188594,NaN,890.164309,890.240292,890.723677,891.936187,981.366041,1096.449849,1157.316312,...,56305.621978,36.133481,-94.732436,NaN,NaN,214.733727,0.700225,0.612598,0.473612,179.317488
std,0.500005,7.439172,NaN,439.723228,439.598307,439.085968,438.015558,473.822787,530.890820,545.697242,...,29432.537945,3.845874,16.877592,NaN,NaN,34.555397,0.458164,0.487162,0.499308,57.278138
min,0.000000,1995.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1731.000000,21.340583,-157.951340,NaN,NaN,152.383333,0.000000,0.000000,0.000000,88.070000
25%,0.000000,2003.000000,NaN,588.000000,588.000000,588.000000,588.000000,662.000000,735.000000,779.460000,...,28352.000000,33.750441,-110.848722,NaN,NaN,183.958333,0.000000,0.000000,0.000000,139.560000
50%,0.500000,2009.000000,NaN,843.000000,843.000000,843.000000,843.000000,939.000000,1041.000000,1110.000000,...,45433.000000,35.314570,-87.669699,NaN,NaN,215.302500,1.000000,1.000000,0.000000,167.900000
75%,1.000000,2015.000000,NaN,1117.000000,1117.000000,1117.000000,1117.000000,1221.000000,1377.000000,1444.000000,...,84120.000000,38.559097,-79.479453,NaN,NaN,237.017000,1.000000,1.000000,1.000000,204.710000


In [9]:
# count number of rows that contain null values
null_rows = base_df.isnull().any(axis=1).sum()
print(f"There are {null_rows} rows with null values out of {base_df.shape[0]} total rows for {100 * null_rows / base_df.shape[0]:.2f} percent of the data.")

There are 23130 rows with null values out of 47976 total rows for 48.21 percent of the data.


In [10]:
# count the number of null values per column
null_column = base_df.isnull().sum()
print("Number of null values per column:")
print(null_column)

Number of null values per column:
Dependents                   0
Year                         0
MHA                          0
E1                          46
E2                          46
E3                          46
E4                          46
E5                          46
E6                          46
E7                          46
E8                          46
E9                          46
W1                          46
W2                          46
W3                          46
W4                          46
W5                          46
O1E                         46
O2E                         46
O3E                         46
O1                          46
O2                          46
O3                          46
O4                          46
O5                          46
O6                          46
O7                        1730
O8                       22610
O9                       22610
O10                      21458
ZIP                          0
Latit

From these results we can see that there are almost 50% of our rows that contain null values, which is an extreme amount. Further investigating reveals that the main culprit are three columns (O8, O9, and O10) that are the main problems. The majority of the data have similar null values per column, which seem very likely to be correlated. First we should investigate the primary null columns.

In [12]:
# view data where none of the rows are null
no_nulls_df = base_df[base_df.notnull().all(axis=1)]
no_nulls_df.head()

,Dependents,Year,MHA,E1,E2,E3,E4,E5,E6,E7,...,ZIP,Latitude,Longitude,CITY,STATE,Average CPI,Senate Majority Party,House Majority Party,President Party,Index (NSA)
18676,1,2007,AK404,1525.0,1525.0,1525.0,1525.0,1625.0,1704.0,1758.0,...,99505,61.266470,-149.641223,ANCHORAGE,AK,207.342417,1,1,1,204.13
18677,1,2007,AK404,1525.0,1525.0,1525.0,1525.0,1625.0,1704.0,1758.0,...,99505,61.266470,-149.641223,ANCHORAGE,AK,207.342417,1,1,1,182.15
18678,1,2007,AK404,1525.0,1525.0,1525.0,1525.0,1625.0,1704.0,1758.0,...,99505,61.266470,-149.641223,ANCHORAGE,AK,207.342417,0,0,1,204.13
18679,1,2007,AK404,1525.0,1525.0,1525.0,1525.0,1625.0,1704.0,1758.0,...,99505,61.266470,-149.641223,ANCHORAGE,AK,207.342417,0,0,1,182.15
18688,1,2007,AL001,657.0,657.0,657.0,657.0,700.0,850.0,888.0,...,36201,33.652278,-85.968851,ANNISTON/FORT MCCLELLAN,AL,207.342417,1,1,1,167.40


In [13]:
# examine the problematic columns
bad_cols = ["O7", "O8", "O9", "O10"]
no_nulls_df[bad_cols].describe()

,O7,O8,O9,O10
count,24846.000000,24846.000000,24846.000000,24846.000000
mean,1932.652016,1932.652016,1932.652016,1932.652016
std,586.912080,586.912080,586.912080,586.912080
min,846.000000,846.000000,846.000000,846.000000
25%,1521.000000,1521.000000,1521.000000,1521.000000
50%,1836.000000,1836.000000,1836.000000,1836.000000
75%,2217.000000,2217.000000,2217.000000,2217.000000
max,5286.000000,5286.000000,5286.000000,5286.000000


*Note:* `describe()` returns exactly equal results for each, implying each column are exactly the same.

In [14]:
# test if the columns are equal
for col in bad_cols:
    match = no_nulls_df["O7"].equals(no_nulls_df[col])
    print(f"Does column O7 equal {col}: {match}")

Does column O7 equal O7: True
Does column O7 equal O8: True
Does column O7 equal O9: True
Does column O7 equal O10: True


Since each column is identical in the no-nulls dataframe, we can reasonably assume that they should be equal in all cases. We should be able to safely eliminate columns O8, O9, and O10.

Additionally, the Index (NSA) column has some null values, but that data is limited to what was available for each 3-digit ZIP code. There is no way for us to impute that information reliably, so any rows with null values in that column should also be dropped.

## 2.(a) Data Cleaning

We have identified two different cleaning steps to take:
- Remove columns O8, O9, and O10
- Drop rows with null values in the Index (NSA) column

In [15]:
# drop the columns
del_cols = ["O8", "O9", "O10"]
cycle1_df = base_df.drop(columns=del_cols)
cycle1_df.shape

(47976, 37)

In [16]:
# drop rows with null HPI values
cycle1_df = cycle1_df[cycle1_df["Index (NSA)"].notnull()]
cycle1_df.shape

(46962, 37)

In [18]:
print(f"{base_df.shape[1] - cycle1_df.shape[1]} total columns removed.")
print(f"{base_df.shape[0] - cycle1_df.shape[0]} total rows removed.")

3 total columns removed.
1014 total rows removed.


Both of these total removed items match expectations. Three columns are deleted and the max number of null values were removed for the Index (NSA) column.

## 1.(b) Data Exploration

Second iteration of exploration after the first cycle.

In [19]:
# count number of rows that contain null values
null_rows = cycle1_df.isnull().any(axis=1).sum()
print(f"There are {null_rows} rows with null values out of {cycle1_df.shape[0]} total rows for {100 * null_rows / cycle1_df.shape[0]:.2f} percent of the data.")

There are 1652 rows with null values out of 46962 total rows for 3.52 percent of the data.


In [20]:
# count the number of null values per column
null_column = cycle1_df.isnull().sum()
print("Number of null values per column:")
print(null_column)

Number of null values per column:
Dependents                  0
Year                        0
MHA                         0
E1                         46
E2                         46
E3                         46
E4                         46
E5                         46
E6                         46
E7                         46
E8                         46
E9                         46
W1                         46
W2                         46
W3                         46
W4                         46
W5                         46
O1E                        46
O2E                        46
O3E                        46
O1                         46
O2                         46
O3                         46
O4                         46
O5                         46
O6                         46
O7                       1652
ZIP                         0
Latitude                    0
Longitude                   0
CITY                        0
STATE                       0
Averag

We can see that O7 still has a large number of null values, which is driving the number of rows with null values, which means there is some overlap with the other 46 null values. We can examine the individual null rows based on an arbitrary column. We'll use E5.

In [21]:
# view data where E5 is null
null_E5_df = cycle1_df[cycle1_df["E5"].isnull()]
null_E5_df.head()

,Dependents,Year,MHA,E1,E2,E3,E4,E5,E6,E7,...,ZIP,Latitude,Longitude,CITY,STATE,Average CPI,Senate Majority Party,House Majority Party,President Party,Index (NSA)
1336,0,1996,ZZ560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37355,35.363999,-86.075033,COUNTY COST GROUP 560,CCG,156.85,1,1,0,105.18
1337,0,1996,ZZ560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37355,35.363999,-86.075033,COUNTY COST GROUP 560,CCG,156.85,1,1,0,105.82
1338,0,1996,ZZ560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37355,35.363999,-86.075033,COUNTY COST GROUP 560,CCG,156.85,1,1,0,106.08
1339,0,1996,ZZ560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37355,35.363999,-86.075033,COUNTY COST GROUP 560,CCG,156.85,1,1,0,103.64
1340,0,1996,ZZ560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37355,35.363999,-86.075033,COUNTY COST GROUP 560,CCG,156.85,1,1,0,103.90


In [22]:
# use the describe method to identify any other correlation
null_E5_df.describe(include="all")

,Dependents,Year,MHA,E1,E2,E3,E4,E5,E6,E7,...,ZIP,Latitude,Longitude,CITY,STATE,Average CPI,Senate Majority Party,House Majority Party,President Party,Index (NSA)
count,46.0,46.0,46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,46.0,4.600000e+01,4.600000e+01,46,46,4.600000e+01,46.0,46.0,46.0,46.000000
unique,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,ZZ560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,COUNTY COST GROUP 560,CCG,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,46,46,NaN,NaN,NaN,NaN,NaN
mean,0.0,1996.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37355.0,3.536400e+01,-8.607503e+01,NaN,NaN,1.568500e+02,1.0,1.0,0.0,105.953043
std,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,4.310366e-14,5.747154e-14,NaN,NaN,1.436789e-13,0.0,0.0,0.0,2.418976
min,0.0,1996.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37355.0,3.536400e+01,-8.607503e+01,NaN,NaN,1.568500e+02,1.0,1.0,0.0,103.080000
25%,0.0,1996.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37355.0,3.536400e+01,-8.607503e+01,NaN,NaN,1.568500e+02,1.0,1.0,0.0,104.195000
50%,0.0,1996.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37355.0,3.536400e+01,-8.607503e+01,NaN,NaN,1.568500e+02,1.0,1.0,0.0,105.485000
75%,0.0,1996.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37355.0,3.536400e+01,-8.607503e+01,NaN,NaN,1.568500e+02,1.0,1.0,0.0,106.717500


From the `describe()` method we can see that each of these null values are associated with the same MHA: ZZ560. The label for this MHA is County Cost Group 560, these are typically placeholder values, or are used for very low-population regions. This data could be eliminated, however it is only for the year 1996. This means that there *is* data for this MHA for the other years. This may be a better candidate for data imputation instead of deletion.

In [23]:
# examine the null values for O7
null_O7_df = cycle1_df[cycle1_df["O7"].isnull()]
null_O7_df.head()

,Dependents,Year,MHA,E1,E2,E3,E4,E5,E6,E7,...,ZIP,Latitude,Longitude,CITY,STATE,Average CPI,Senate Majority Party,House Majority Party,President Party,Index (NSA)
0,1,1995,AK404,343.72,336.38,342.40,389.17,478.92,545.71,612.63,...,99505,61.266470,-149.641223,ANCHORAGE,AK,152.383333,0,0,0,100.0
1,1,1995,AK404,343.72,336.38,342.40,389.17,478.92,545.71,612.63,...,99505,61.266470,-149.641223,ANCHORAGE,AK,152.383333,1,1,0,100.0
2,0,1995,AK404,192.42,211.35,252.19,271.72,334.33,371.40,425.62,...,99505,61.266470,-149.641223,ANCHORAGE,AK,152.383333,0,0,0,100.0
3,0,1995,AK404,192.42,211.35,252.19,271.72,334.33,371.40,425.62,...,99505,61.266470,-149.641223,ANCHORAGE,AK,152.383333,1,1,0,100.0
8,1,1995,AL001,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,36201,33.652278,-85.968851,ANNISTON/FORT MCCLELLAN,AL,152.383333,0,0,0,100.0


In [24]:
# use the describe method to identify any other correlation
null_O7_df.describe(include="all")

,Dependents,Year,MHA,E1,E2,E3,E4,E5,E6,E7,...,ZIP,Latitude,Longitude,CITY,STATE,Average CPI,Senate Majority Party,House Majority Party,President Party,Index (NSA)
count,1652.000000,1652.000000,1652,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,...,1652.000000,1652.000000,1652.000000,1652,1652,1652.000000,1652.000000,1652.000000,1652.0,1652.000000
unique,NaN,NaN,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,106,42,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,ZZ510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,COUNTY COST GROUP 510,CCG,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,250,710,NaN,NaN,NaN,NaN,NaN
mean,0.500000,1995.743341,NaN,70.171357,72.329072,72.311071,76.527161,89.242995,101.994371,118.847615,...,55662.629540,36.149302,-94.724479,NaN,NaN,155.703592,0.871671,0.871671,0.0,104.179831
std,0.500151,0.436922,NaN,76.344966,77.813998,80.683334,85.076834,94.364153,106.269760,120.089145,...,29466.815876,4.167392,17.115917,NaN,NaN,1.951583,0.334557,0.334557,0.0,3.223231
min,0.000000,1995.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1731.000000,21.340583,-157.951340,NaN,NaN,152.383333,0.000000,0.000000,0.0,90.890000
25%,0.000000,1995.000000,NaN,6.027500,3.830000,0.000000,0.210000,10.380000,6.902500,12.160000,...,28352.000000,33.652278,-111.178514,NaN,NaN,152.383333,1.000000,1.000000,0.0,100.000000
50%,0.500000,1996.000000,NaN,48.390000,54.320000,52.550000,55.550000,67.945000,78.360000,94.020000,...,42254.000000,35.314570,-87.669699,NaN,NaN,156.850000,1.000000,1.000000,0.0,104.665000
75%,1.000000,1996.000000,NaN,98.692500,94.372500,102.000000,107.812500,133.990000,151.890000,176.400000,...,84120.000000,38.805747,-79.479453,NaN,NaN,156.850000,1.000000,1.000000,0.0,106.230000


Using the `describe()` method on the O7 null rows shows another trend. The data for this grade is null for years 1995 and 1996, but only across 106 unique MHAs out of the 120 unique MHAs from the base data. This lead to some additional research about the beginning of the BAH program.

November 18, 1997, the Basic Allowance for Housing (BAH) program was written into law, and went into effect in 1998. Prior to that time servicemembers received a Basic Allowance for Quarters (BAQ) which was not regionally adjusted. Therefore any data prior to 1998 should be excluded regardless.

>DIVISION A-DEPARTMENT OF DEFENSE AUTHORIZATIONS TITLE I-PROCUREMENT Subtitle A-Authorization of Appropriations. (n.d.). Retrieved February 16, 2025, from https://www.govinfo.gov/content/pkg/PLAW-105publ85/pdf/PLAW-105publ85.pdf

## 2.(b) Data Cleaning

Second iteration of cleaning the data.

In [25]:
# remove data from 1995 through 1997
cycle2_df = cycle1_df[cycle1_df["Year"] >= 1998]
cycle2_df.shape

(45310, 37)

In [26]:
# compare to previous iteration
print(f"Total columns removed (expected = 0): {cycle1_df.shape[1] - cycle2_df.shape[1]}")
print(f"Total rows removed: {cycle1_df.shape[0] - cycle2_df.shape[0]}")

Total columns removed (expected = 0): 0
Total rows removed: 1652


In [27]:
# examine nulls to see if more cleaning is required
null_column = cycle2_df.isnull().sum()
print("Number of null values per column:")
print(null_column)

Number of null values per column:
Dependents               0
Year                     0
MHA                      0
E1                       0
E2                       0
E3                       0
E4                       0
E5                       0
E6                       0
E7                       0
E8                       0
E9                       0
W1                       0
W2                       0
W3                       0
W4                       0
W5                       0
O1E                      0
O2E                      0
O3E                      0
O1                       0
O2                       0
O3                       0
O4                       0
O5                       0
O6                       0
O7                       0
ZIP                      0
Latitude                 0
Longitude                0
CITY                     0
STATE                    0
Average CPI              0
Senate Majority Party    0
House Majority Party     0
President Party      

We can see that we have eliminated all null values.

## 3. Data Imputation

No empty data remains so no imputation methods are necessary.

## 4. Merge Final Data Set

In [28]:
# save final dataframe
clean_file = os.path.join(clean_dir, "clean_df.csv")
cycle2_df.to_csv(clean_file, index=False)